In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql

In [2]:
from shared_utils import webmap_utils, catalog_utils
from calitp_data_analysis.geography_utils import WGS84, CA_NAD83Albers_m

# Check GTFS station entrance coverage

* Mainly BART and LA Metro

In [3]:
df = query_sql('select distinct gtfs_dataset_name from cal-itp-data-infra.mart_gtfs.fct_daily_schedule_feeds where date = "2025-08-18"')

In [6]:
df[df.gtfs_dataset_name.str.contains('Muni')]

,gtfs_dataset_name
131,Bay Area 511 Muni Schedule


In [25]:
query = '''
SELECT t1.gtfs_dataset_name, t1.date, t2.feed_key, t2.stop_id,
t2.parent_station, t2.stop_code, t2.stop_name, t2.stop_desc, t2.location_type, t2.pt_geom
FROM cal-itp-data-infra.mart_gtfs.fct_daily_schedule_feeds AS t1
INNER JOIN cal-itp-data-infra.mart_gtfs.dim_stops AS t2
ON t1.feed_key = t2.feed_key
WHERE t1.gtfs_dataset_name IN ("LA Metro Rail Schedule", "Bay Area 511 BART Schedule", "Metrolink Schedule",
"Bay Area 511 Sonoma-Marin Area Rail Transit Schedule", "Sacramento Schedule", "San Diego Schedule", "Bay Area 511 Muni Schedule")
AND t1.date = '2026-01-07' AND t2.location_type != 0.0
'''

In [26]:
df2 = query_sql(query)

In [27]:
gdf = gpd.GeoDataFrame(df2, geometry=gpd.GeoSeries.from_wkt(df2.pt_geom), crs=WGS84)

In [28]:
gdf

,gtfs_dataset_name,date,feed_key,stop_id,parent_station,stop_code,stop_name,stop_desc,location_type,pt_geom,geometry
0,San Diego Schedule,2026-01-07,6ec100ffba38b0007881f35ed4240e72,amptro,None,None,America Plaza Trolley Station,None,1,POINT(-117.16835451 32.7170022),POINT (-117.16835 32.71700)
1,San Diego Schedule,2026-01-07,6ec100ffba38b0007881f35ed4240e72,iritrW,None,None,Iris Ave Transit Center West,None,1,POINT(-117.0672329 32.56968604),POINT (-117.06723 32.56969)
2,San Diego Schedule,2026-01-07,6ec100ffba38b0007881f35ed4240e72,masS,None,None,Massachusetts Ave Station,None,1,POINT(-117.03958489 32.72352489),POINT (-117.03958 32.72352)
3,San Diego Schedule,2026-01-07,6ec100ffba38b0007881f35ed4240e72,harS,None,None,Harborside Station,None,1,POINT(-117.13280415 32.69126696),POINT (-117.13280 32.69127)
4,San Diego Schedule,2026-01-07,6ec100ffba38b0007881f35ed4240e72,avlS,None,None,UCSD Health East Station,None,1,POINT(-117.05683205 32.77736594),POINT (-117.05683 32.77737)
...,...,...,...,...,...,...,...,...,...,...,...
631,LA Metro Rail Schedule,2026-01-07,98ebd9ff823d7be7e37665c0674b0722,801100S,None,801100S,Glendora Station,None,1,POINT(-117.8663 34.1319),POINT (-117.86630 34.13190)
632,LA Metro Rail Schedule,2026-01-07,98ebd9ff823d7be7e37665c0674b0722,80313S,None,80313S,Lakewood Blvd Station,None,1,POINT(-118.1406 33.91307),POINT (-118.14060 33.91307)
633,LA Metro Rail Schedule,2026-01-07,98ebd9ff823d7be7e37665c0674b0722,80410A,80410S,80410A,Chinatown Station - Northeast Entrance,None,2,POINT(-118.23573 34.06399),POINT (-118.23573 34.06399)
634,LA Metro Rail Schedule,2026-01-07,98ebd9ff823d7be7e37665c0674b0722,80301B,80301S,80301B,Redondo Beach Station - North Stairs,None,2,POINT(-118.369617 33.894967),POINT (-118.36962 33.89497)


In [29]:
gdf.gtfs_dataset_name.unique()

array(['San Diego Schedule', 'Bay Area 511 BART Schedule',
       'Bay Area 511 Sonoma-Marin Area Rail Transit Schedule',
       'LA Metro Rail Schedule'], dtype=object)

In [19]:
gdf[gdf.gtfs_dataset_name == 'Bay Area 511 Muni Schedule'].parent_station.unique()

array([None], dtype=object)

In [31]:
# gdf.drop(columns=['date']).explore(tiles='CartoDB Positron')